In [3]:
def get_avg_ratio(numbers):
    average = sum(numbers) / len(numbers)
    scaled = [x / average for x in numbers]
    scaled.sort(reverse=True)
    return scaled

In [5]:
lengths = [63, 73, 72, 60, 67, 66, 71, 61, 72, 70]

longest, *middle, shortest = get_avg_ratio(lengths)

print(f'Longest: {longest:>4.0%}')
print(f'Shortest: {shortest:>4.0%}')

Longest: 108%
Shortest:  89%


### Item 20 Prefer Raising Exceptions to Returning *None*

In [6]:
def careful_divide(a,b):
    try:
        return True, a / b
    except ZeroDivisionError:
        return False, None
    

In [11]:
success, result = careful_divide(0,5)

if not success:
    print('invalid inputs')
else:
    print('Result is %.1f' % result)

Result is 0.0


In [12]:
def careful_division(a: float, b: float) -> float:
    """Divides a by b.
    Raises:
        ValueError: When the inputs cannot be divided."""
    
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('Invalid inputs')

In [15]:
careful_division(0,5)

0.0